In [1]:
import pandas as pd
from sklearn.model_selection import GroupKFold
import numpy as np

In [2]:
train_descriptions = pd.read_csv(
        "../../../input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv"
    )
clean_keypoints = pd.read_csv(
    "../../../input/lumbar-coordinate-pretraining-dataset/coords_rsna_improved.csv",
    index_col=0,
)
train_coord_df = pd.concat(
    [
        clean_keypoints[
            (clean_keypoints.condition == "Spinal Canal Stenosis")
            & (clean_keypoints.side == "R")
        ],
        clean_keypoints[~(clean_keypoints.condition == "Spinal Canal Stenosis")],
    ]
)
train_coord_df = train_coord_df.merge(
    train_descriptions, on=["series_id", "study_id"], how="left"
)
train_df = pd.read_csv(
    "../../../input/rsna-2024-lumbar-spine-degenerative-classification/train.csv"
)
train_df["fold_id"] = -1
for i, (train_index, valid_index) in enumerate(
    GroupKFold(n_splits=5).split(
        train_df, np.arange(len(train_df)), train_df.study_id
    )
):
    train_df.loc[valid_index, "fold_id"] = i
train_coord_df = train_coord_df.merge(
    train_df.loc[:, ["study_id", "fold_id"]], on=["study_id"], how="left"
)
new_train_coord_df = train_coord_df.sort_values(
    by=["study_id", "series_id", "level", "condition"]
).reset_index(drop=True)
wrong_series_ids = [3951475160]
new_train_coord_df = new_train_coord_df[
    ~new_train_coord_df.series_id.isin(wrong_series_ids)
]

In [3]:
new_train_coord_df.groupby("series_id").count().relative_x.value_counts()

relative_x
10    3459
5     2278
6      201
4      137
8       82
2       42
3       40
1       20
9       17
7       14
Name: count, dtype: int64

In [4]:
new_train_coord_df[new_train_coord_df.series_description == "Sagittal T2/STIR"].groupby("series_id").count().relative_x.value_counts()

relative_x
5    1973
Name: count, dtype: int64

In [5]:
new_train_coord_df[new_train_coord_df.series_description == "Sagittal T1"].groupby("series_id").count().relative_x.value_counts()

relative_x
10    1978
Name: count, dtype: int64

In [13]:
new_train_coord_df[new_train_coord_df.series_description == "Sagittal T1"].groupby("series_id").count()[new_train_coord_df[new_train_coord_df.series_description == "Sagittal T1"].groupby("series_id").count().relative_x == 15]
new_train_coord_df[new_train_coord_df.series_id==3951475160]

,series_id,relative_x,relative_y,condition,side,instance_number,study_id,level,series_description,fold_id
25130,3951475160,0.569617,0.269723,Left Neural Foraminal Narrowing,L,11,3637444890,L1/L2,Sagittal T1,3
25131,3951475160,0.573530,0.268595,Right Neural Foraminal Narrowing,R,5,3637444890,L1/L2,Sagittal T1,3
25132,3951475160,0.559511,0.285685,Spinal Canal Stenosis,R,8,3637444890,L1/L2,Sagittal T1,3
25133,3951475160,0.537803,0.364341,Left Neural Foraminal Narrowing,L,12,3637444890,L2/L3,Sagittal T1,3
25134,3951475160,0.537717,0.374656,Right Neural Foraminal Narrowing,R,5,3637444890,L2/L3,Sagittal T1,3
25135,3951475160,0.532263,0.391846,Spinal Canal Stenosis,R,8,3637444890,L2/L3,Sagittal T1,3
25136,3951475160,0.515337,0.472674,Left Neural Foraminal Narrowing,L,12,3637444890,L3/L4,Sagittal T1,3
25137,3951475160,0.511547,0.482094,Right Neural Foraminal Narrowing,R,5,3637444890,L3/L4,Sagittal T1,3
25138,3951475160,0.524089,0.506423,Spinal Canal Stenosis,R,8,3637444890,L3/L4,Sagittal T1,3
25139,3951475160,0.509237,0.589001,Left Neural Foraminal Narrowing,L,12,3637444890,L4/L5,Sagittal T1,3
